In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time 
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from procurement_lib import GoogleSheet

In [2]:
df_gtin = GoogleSheet("1TBAwmxAoqg6FX79d7wLu3DVtIiOb6gRfkN5fMA_tLNc")
df_gtin = df_gtin.get_as_dataframe('Hoja 1')

In [3]:
query = """
SELECT DISTINCT
    CASE 
    WHEN state = 'Grande SP' THEN 'SPO'
    WHEN state = 'SP Interior' THEN 'VCP'
    WHEN state = 'MG' THEN 'BHZ'
    WHEN state = 'PR' THEN 'CWB'
    ELSE 'NA' END AS site_code,
    i.price_date AS quotation_date,
    CAST(REPLACE(i.price_date, '-', '') AS INTEGER) AS dim_quotation_date,
    i.store_type || ' - ' || i.store_identifier AS competitor_name,
    i.store_type || ' - ' || i.store AS store,
    gtin,
    ROUND(AVG(COALESCE(
        REPLACE(NULLIF(i.regular_price::VARCHAR, ''), ' ', '0')::FLOAT,
        REPLACE(NULLIF(i.price_paid::VARCHAR, ''), ' ', '0')::FLOAT,
        REPLACE(NULLIF(i.price_paid_per_standard_unit_of_measure::VARCHAR, ''), ' ', '0')::FLOAT,
        0.0
    )),2)::FLOAT AS price
    
FROM lnd_ops.infoprice_product_scraper i
WHERE store_type IN ('Atacado')
 AND gtin IN {skus}
GROUP BY 1,2,3,4,5,6
""".format(skus=tuple(df_gtin.GTIN.unique().tolist()))

df = read_connection_data_warehouse.run_read_prod_query(query)

df['quotation_date'] = pd.to_datetime(df['quotation_date'])
df[['gtin']] = df[['gtin']].astype('float')

In [4]:
df2 = pd.merge(df, df_gtin, left_on=['gtin','site_code'], right_on=['GTIN','city'], how='left')
df2 = df2.drop(['city', 'GTIN'], axis=1)
df2 = df2.dropna().reset_index(drop=True)

df2['lifetime'] = 8

In [5]:
df2.source_id.value_counts()

105599.0    21936
124893.0    14208
73456.0     13733
124405.0    12132
217683.0    11866
            ...  
643817.0        1
531043.0        1
356957.0        1
581671.0        1
418949.0        1
Name: source_id, Length: 1728, dtype: int64

In [6]:
# Step 1: Group the data by store, quotation_date, and source_id (product)
grouped = df2.groupby(['site_code','store', 'quotation_date', 'source_id'])
grouped

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2290306 entries, 0 to 2290305
Data columns (total 10 columns):
 #   Column              Dtype         
---  ------              -----         
 0   site_code           object        
 1   quotation_date      datetime64[ns]
 2   dim_quotation_date  int64         
 3   competitor_name     object        
 4   store               object        
 5   gtin                float64       
 6   price               float64       
 7   source_id           float64       
 8   product_name        object        
 9   lifetime            int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 174.7+ MB


In [8]:
df3 = df2.groupby(['site_code','store','quotation_date','price','source_id']).count().reset_index()

In [9]:
r = (df3.groupby(['site_code','store','quotation_date','source_id']).gtin.max() / df3.groupby(['site_code','store','quotation_date','source_id']).gtin.sum()).reset_index()

In [10]:
r.loc[(r.source_id == 73456) & (r.quotation_date == '2024-02-10')].head()

,site_code,store,quotation_date,source_id,gtin
73502,SPO,Atacado - Akki Atacadista,2024-02-10,73456.0,0.750000
80963,SPO,Atacado - Assai Atacadista,2024-02-10,73456.0,0.901961
110008,SPO,Atacado - Atacadao Dia A Dia,2024-02-10,73456.0,1.000000
113062,SPO,Atacado - Atacadista Roldao,2024-02-10,73456.0,0.550000
143454,SPO,Atacado - Atacadão,2024-02-10,73456.0,0.981132


In [11]:
f = (r.groupby(['site_code','store','source_id']).gtin.mean().reset_index()).groupby(['site_code','store']).gtin.mean().reset_index()

In [18]:
df_axu = df2.groupby('store').agg({'competitor_name':lambda x: x.nunique()}).reset_index()

In [19]:
df_f = f.merge(df_axu, on=['store'],how='left')

In [20]:
df_f

,site_code,store,gtin,competitor_name
0,BHZ,Atacado - Aliar Distribuidora,1.000000,1
1,BHZ,Atacado - Alimentar Foods Frigorifico,1.000000,1
2,BHZ,Atacado - Almeida Atacarejo,1.000000,1
3,BHZ,Atacado - Ambbar Atacadista,1.000000,1
4,BHZ,Atacado - Apoio Mineiro,0.926860,29
...,...,...,...,...
231,VCP,Atacado - Supermercado Leal Do Vale,1.000000,1
232,VCP,Atacado - Tenda,0.726622,55
233,VCP,Atacado - Tonin,0.996958,13
234,VCP,Atacado - Zaragoza,0.996318,4


In [ ]:

# Step 3: Identify the frequency of the most common price for each group
max_freq = price_counts.max(axis=1)

# Step 4: Calculate the percentage for each group
total_counts = grouped.size()
percentage_freq = (max_freq / total_counts) * 100

# Step 5: Average these percentages for each store
average_percentage = percentage_freq.groupby(level=0).mean()

# Display the result
average_percentage

In [31]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Load the data (ensure df2 is loaded appropriately)

# Convert the 'quotation_date' to datetime if it's not already
df2['quotation_date'] = pd.to_datetime(df2['quotation_date'])

# Extract the unique quotation dates for the dropdown
dates = sorted(df2['quotation_date'].dt.date.unique())

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=dates,
    value=dates[0],
    description='Date:',
    style={'description_width': 'initial'}
)

def plot_price_distribution(selected_date):
    selected_date = pd.to_datetime(selected_date)
    filtered_df2 = df2[(df2['source_id'] == 73456) & (df2['quotation_date'].dt.date == selected_date.date())]
    
    # Create the figure
    fig = go.Figure(layout=go.Layout(height=800))
    
    # Add histogram traces for each store
    for store in filtered_df2['store'].unique():
        store_data = filtered_df2[filtered_df2['store'] == store]
        fig.add_trace(go.Histogram(
            x=store_data['price'],
            name=store,
            nbinsx=30,
            opacity=0.75
        ))
    
    # Update the layout for stacked histogram
    fig.update_layout(
        barmode='stack',
        title=f'Price Distribution for {selected_date.strftime("%Y-%m-%d")} - {filtered_df2.product_name.values[0]}',
        xaxis_title='Price',
        yaxis_title='Frequency',
        legend_title='Store',
        legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5)
    )
    
    fig.show()

# Use the dropdown value to update the plot
widgets.interact(plot_price_distribution, selected_date=dropdown)

interactive(children=(Dropdown(description='Date:', options=(datetime.date(2024, 2, 10), datetime.date(2024, 2…

<function __main__.plot_price_distribution(selected_date)>